In [1]:
import json
import numpy as np
import pandas as pd
import os
import subprocess

In [15]:
for i in range(89,88,-1):
    print i
    print "copying locally..."
    #subprocess.check_call("./copydata.sh "+str(i), shell=True)

    print "making csv..."
    pathtodata = '/media/anne/LACIE SHARE/DataScienceFinalProject/'
    #pathtodata = 'data/'
    make_csv(i, pathtodata)

    print "deleting "+str(i)+" locally..."
    #subprocess.check_call("./rmdata.sh "+str(i), shell=True)

89
copying locally...
making csv...
1965 1 201
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h1/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h2/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h3/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h4/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h5/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h6/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h7/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h8/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h9/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h10/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h11/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89/votes/1965/h12/data.json
/media/anne/LACIE SHARE/DataScienceFinalProject/89

In [2]:
def convert_str_to_float(string):
    num, denom = string.split('/')
    return float(num)/float(denom)

def pad_dict(vote_dict):
    for key in vote_dict:
        if len(vote_dict[key]) != len(vote_dict['isAmendment']):
            vote_dict[key] = vote_dict[key] + [np.nan]*(len(vote_dict['isAmendment']) - len(vote_dict[key]))
    return vote_dict

In [3]:
def make_csv(congress_no, pathtodata):
    content_list = []

    for content in os.listdir(pathtodata+str(congress_no)+ "/votes"):
        if content != '.DS_Store':
            content_list.append(content)

    def get_vote_nums(path):
        housevotes = []
        senvotes = []
        for folder in os.listdir(path):
            if folder[0] == 'h':
                housevotes.append(int(folder[1:]))
            elif folder[0] == 's':
                senvotes.append(int(folder[1:]))   
        housevotes.sort()
        senvotes.sort()
        if len(housevotes) == 0:
            housevotes = [0]
        if len(senvotes) == 0:
            senvotes = [0]
        return housevotes[0], housevotes[len(housevotes)-1], senvotes[0], senvotes[len(senvotes)-1]

    house_dict = {}
    senate_dict = {}
    for year in content_list:
        house_count = 0
        senate_count = 0
        path = pathtodata+str(congress_no) + "/votes/" + year
        hstart, hend, sstart, send = get_vote_nums(path)
        print year, hstart, hend
        house = get_vote_data(path+"/h", hstart, hend, house_dict)
        senate = get_vote_data(path+"/s", sstart, send, senate_dict)

    house_dict = pad_dict(house_dict)
    senate_dict = pad_dict(senate_dict)

    df = pd.DataFrame(data=house_dict)
    df.to_csv("csv/" + str(congress_no)+"house.csv", index=False)

    df = pd.DataFrame(data=senate_dict)
    df.to_csv("csv/" + str(congress_no)+"senate.csv", index=False)

In [11]:
votes = ['Yea', 'Nay', 'Present', 'Not Voting']
res2 = []
def get_vote_data(path, start, end, vote_dict):
    for i in range(start, end+1):
        votepath = path + str(i) + "/data.json"
        print votepath
        if os.path.exists(votepath):
            f = open(votepath, 'r')
            vote = json.loads(f.read(), 'utf-8')
            if 'Aye' in vote['votes']:
                vote['votes']['Yea'] = vote['votes']['Aye']
                del vote['votes']['Aye']
            if 'No' in vote['votes']:
                vote['votes']['Nay'] = vote['votes']['No']
                del vote['votes']['No']
            if not any(vote_dict):
                if vote['category'] == 'passage' or vote['category'] == 'amendment' or vote['category'] == 'unknown':
                    res2.append(i)
                    vote_dict['date'] = [vote['date']]
                    if vote['category'] == 'passage' or vote['category'] == 'unknown':
                        if 'bill' not in vote or vote['bill'] == None:
                            vote_dict['title'] = [np.nan]
                        else:
                            vote_dict['title'] = [vote['bill']['type'] + str(vote['bill']['number'])]
                        vote_dict['isAmendment'] = [0]
                    elif vote['category'] == 'amendment' and 'amendment' in vote:
                        if 'bill' not in vote or vote['bill'] == None:
                            vote_dict['title'] = [np.nan]
                        else:
                            vote_dict['title'] = [vote['amendment']['type'][0] + "amdt" + str(vote['amendment']['number']) + " to " + vote['bill']['type'] + str(vote['bill']['number'])]
                        vote_dict['isAmendment'] = [1]
                    else:
                        if 'bill' in vote:
                            vote_dict['title'] = [vote['bill']['type'] + str(vote['bill']['number'])]
                            vote_dict['isAmendment'] = [1]
                        else:
                            vote_dict['title'] = [np.nan]
                            vote_dict['isAmendment'] = [1]
                    if 'result_text' not in vote:
                        vote_dict['result'] = [vote['result']]
                    else:
                        vote_dict['result'] = [vote['result_text']]
                    if vote['requires'] == 'unknown':
                        vote_dict['requires'] = [np.nan]
                    elif vote['requires'] == 'QUORUM':
                        vote_dict['requires'] = [convert_str_to_float("1/2")] # assuming quorum is simple 1/2 majority
                    else:
                        vote_dict['requires'] = [convert_str_to_float(vote['requires'])]
                    for vote_type in votes:
                        if vote_type in vote['votes']:
                            for person in vote['votes'][vote_type]:
                                vote_dict[person['id']] = [vote_type]
            else:
                if vote['category'] == 'passage' or vote['category'] == 'amendment' or vote['category'] == 'unknown': 
                    res2.append(i)
                    vote_dict['date'].append(vote['date'])
                    #vote_dict['billTitle'].append(vote['subject'])
                    if vote['category'] == 'passage' or vote['category'] == 'unknown':
                        if 'bill' not in vote or vote['bill'] == None:
                            vote_dict['title'].append(np.nan)
                        else:
                            vote_dict['title'].append(vote['bill']['type'] + str(vote['bill']['number']))
                        vote_dict['isAmendment'].append(0)
                    elif vote['category'] == 'amendment' and 'amendment' in vote:
                        if 'bill' not in vote or vote['bill'] == None:
                            vote_dict['title'].append(np.nan)
                        else:
                            vote_dict['title'].append(vote['amendment']['type'][0] + "amdt" + str(vote['amendment']['number']) + " to " + vote['bill']['type'] + str(vote['bill']['number']))
                        vote_dict['isAmendment'].append(1)
                    else:
                        if 'bill' in vote:
                            vote_dict['title'].append(vote['bill']['type'] + str(vote['bill']['number']))
                            vote_dict['isAmendment'].append(1)
                        else:
                            vote_dict['title'].append(np.nan)
                            vote_dict['isAmendment'].append(1)
                    if 'result_text' not in vote:
                        vote_dict['result'].append(vote['result'])
                    else:
                        vote_dict['result'].append(vote['result_text'])

                    if vote['requires'] == 'unknown':
                        vote_dict['requires'].append(np.nan)
                    elif vote['requires'] == 'QUORUM':
                        vote_dict['requires'].append(convert_str_to_float("1/2")) # assuming quorum is simple 1/2 majority
                    else:
                        vote_dict['requires'].append(convert_str_to_float(vote['requires']))
                    for vote_type in votes:
                        if vote_type in vote['votes']:
                            for person in vote['votes'][vote_type]:
                                if person == 'VP':
                                    print "The VP voted to break a tie in this vote"
                                elif person['id'] in vote_dict:
                                    vote_dict[person['id']].append(vote_type)
                                else:
                                    vote_dict[person['id']] = [np.nan]*(len(vote_dict['isAmendment'])-1) + [vote_type]
    return vote_dict

# test = get_vote_data('data/113/votes/2014/h', 1, 671, {})
# print len(test['isAmendment'])
# print len(res2)

In [1]:
def get_subjects(isAmendment, title):
    if np.isnan(title):
        return np.nan
    else:
        if isAmendment:
            if len(title.split()) > 1:
                bill = title.split()[2]
            else:
                bill = title
            path = pathtodata + congress_no + "/bills/" + ''.join([i for i in bill if not i.isdigit()]) + "/"



In [64]:
result = ''.join([i for i in 's5' if not i.isdigit()])
print result

s


In [62]:
if np.isnan(np.nan):
    print "Wow"
else:
    print "It works"

Wow
